### Проанализировать датафрейм статистики NBA. (файл nba.csv)


Вывести:
1. Проранжировать бюджет команд на зарплату от самого высокого до самого низкого.(выполнено)
2. 10 наиболее высокооплачиваемых игроков в лиге.(выполнено)
3. 3 наиболее высооплачиваемых игроков в каждой команде. (выполнено)
4. Предложить первые пятерки каждой команды на основе их зарплаты (учитывайте, 
   что в первой пятерке на каждой позиции один игрок (см. рисунок)).
5. Самая молодая и самая возростная команда в лиге. (выполнено)
6. Наиболее часто используемый номер игрока в лиге. (выполнено)
7. Имя и рост самого высокого и самого низкого игрока в лиге в метрической системе.(выполнено)

   
P.S. Возможные пропуски в зарплатной ведомости заполнить медианным значением зарплаты команды игрока с пропуском.

In [2]:
import pandas as pd
import numpy as np

In [56]:
data = pd.read_csv('nba.csv', delimiter = ',')
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)
#Последняя строка пустая
cleanData = data.drop([457][0])
cleanData

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0
...,...,...,...,...,...,...,...,...,...
452,Trey Lyles,Utah Jazz,41.0,PF,20.0,6-10,234.0,Kentucky,2239800.0
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0


In [74]:
#10 наиболее высокооплачиваемых игроков в лиге.
richestPlayers = cleanData.nlargest(10, ['Salary'])
richestPlayers[['Name','Salary']]

,Name,Salary
109,Kobe Bryant,25000000.0
169,LeBron James,22970500.0
33,Carmelo Anthony,22875000.0
251,Dwight Howard,22359364.0
339,Chris Bosh,22192730.0
100,Chris Paul,21468695.0
414,Kevin Durant,20158622.0
164,Derrick Rose,20093064.0
349,Dwyane Wade,20000000.0
23,Brook Lopez,19689000.0


In [58]:
#Самая молодая команда в лиге
meanValues = cleanData.groupby(['Team']).mean().sort_values(by = 'Age')
youngestTeam = meanValues.Age[0]
oldestTeam = meanValues.Age[-1]
meanValues.loc[meanValues['Age'] == youngestTeam] 


,Number,Age,Weight,Salary
Team,,,,
Utah Jazz,17.866667,24.466667,220.0,4.204006e+06


In [59]:
#Самая возрастная команда в лиге
meanValues.loc[meanValues['Age'] == oldestTeam]

,Number,Age,Weight,Salary
Team,,,,
San Antonio Spurs,17.933333,31.6,223.933333,5.629516e+06


In [60]:
#Наиболее часто используемый номер игрока в лиге
cleanData['Number'].value_counts().head(1)

5.0    23
Name: Number, dtype: int64

In [62]:
#Переводим рост игроков в метрическую систему
cleanData.Height = cleanData.Height.apply(lambda el: round(int(el.split('-')[0]) * 30.48 + int(el.split('-')[1])* 2.54, 2))
sortedHeight = cleanData[['Height','Name']].sort_values(by='Height')

In [63]:
#Рост и имя самого низкого игрока
sortedHeight.iloc[0] 


Height           175.26
Name      Isaiah Thomas
Name: 11, dtype: object

In [64]:
#Рост и имя самого высокого игрока
sortedHeight.iloc[-1]

Height              220.98
Name      Boban Marjanovic
Name: 302, dtype: object

In [75]:
#Количество игроков с пропущенным значением зарплаты
cleanData.Salary.isnull().sum()

0

In [76]:
#Заполнение пропусков в графе зарплаты
teams = cleanData.Team.unique()
for i in teams:
    cleanData = cleanData.fillna({'Salary':cleanData[ cleanData.Team == i ]['Salary'].mean()})   

In [77]:
#Ранжирование бюджета команд на зарплату от самого высокого до самого низкого
cleanData.groupby(['Name','Team'])[['Salary']].max().sort_values(by = ['Team', 'Salary'], ascending = [True, False])

,,Salary
Name,Team,
Paul Millsap,Atlanta Hawks,18671659.0
Al Horford,Atlanta Hawks,12000000.0
Tiago Splitter,Atlanta Hawks,9756250.0
Jeff Teague,Atlanta Hawks,8000000.0
Kyle Korver,Atlanta Hawks,5746479.0
...,...,...
Kelly Oubre Jr.,Washington Wizards,1920240.0
Garrett Temple,Washington Wizards,1100602.0
Jarell Eddie,Washington Wizards,561716.0


In [78]:
#3 наиболее высокооплачиваемых игроков в каждой команде 
threePlayer = cleanData.copy()
threePlayer.groupby('Team').apply(lambda threePlayer: threePlayer.sort_values(by = 'Salary', ascending = False).head(3))

Name                Team  Number Position  \
Team                                                                          
Atlanta Hawks      315    Paul Millsap       Atlanta Hawks     4.0       PF   
                   312      Al Horford       Atlanta Hawks    15.0        C   
                   321  Tiago Splitter       Atlanta Hawks    11.0        C   
Boston Celtics     5      Amir Johnson      Boston Celtics    90.0       PF   
                   0     Avery Bradley      Boston Celtics     0.0       PG   
...                                ...                 ...     ...      ...   
Utah Jazz          446  Derrick Favors           Utah Jazz    15.0       PF   
                   444      Alec Burks           Utah Jazz    10.0       SG   
Washington Wizards 382       John Wall  Washington Wizards     2.0       PG   
                   375    Nene Hilario  Washington Wizards    42.0        C   
                   373   Marcin Gortat  Washington Wizards    13.0        C   

                         Age  Height  Weight         College      Salary  
Team                                                                      
Atlanta Hawks      315  31.0  203.20   246.0  Louisiana Tech  18671659.0  
                   312  30.0  208.28   245.0         Florida  12000000.0  
                   321  31.0  210.82   245.0             NaN   9756250.0  
Boston Celtics     5    29.0  205.74   240.0             NaN  12000000.0  
                   0    25.0  187.96   180.0           Texas   7730337.0  
...                      ...     ...     ...             ...         ...  
Utah Jazz          446  24.0  208.28   265.0    Georgia Tech  12000000.0  
                   444  24.0  198.12   214.0        Colorado   9463484.0  
Washington Wizards 382  25.0  193.04   195.0        Kentucky  15851950.0  
                   375  33.0  210.82   250.0             NaN  13000000.0  
                   373  32.0  210.82   240.0             NaN  11217391.0  

[90 rows x 9 columns]

In [79]:
#Предложить первые пятерки каждой команды на основе их зарплаты
fivePlayers = cleanData.copy()
fivePlayers.groupby(['Team','Position']).apply(lambda i: i.loc[i.Salary.idxmax()])

Name                Team  Number  \
Team               Position                                                
Atlanta Hawks      C              Al Horford       Atlanta Hawks    15.0   
                   PF           Paul Millsap       Atlanta Hawks     4.0   
                   PG            Jeff Teague       Atlanta Hawks     0.0   
                   SF        Thabo Sefolosha       Atlanta Hawks    25.0   
                   SG            Kyle Korver       Atlanta Hawks    26.0   
...                                      ...                 ...     ...   
Washington Wizards C            Nene Hilario  Washington Wizards    42.0   
                   PF        Markieff Morris  Washington Wizards     5.0   
                   PG              John Wall  Washington Wizards     2.0   
                   SF        Otto Porter Jr.  Washington Wizards    22.0   
                   SG           Bradley Beal  Washington Wizards     3.0   

                            Position   Age  Height  Weight         College  \
Team               Position                                                  
Atlanta Hawks      C               C  30.0  208.28   245.0         Florida   
                   PF             PF  31.0  203.20   246.0  Louisiana Tech   
                   PG             PG  27.0  187.96   186.0     Wake Forest   
                   SF             SF  32.0  200.66   220.0             NaN   
                   SG             SG  35.0  200.66   212.0       Creighton   
...                              ...   ...     ...     ...             ...   
Washington Wizards C               C  33.0  210.82   250.0             NaN   
                   PF             PF  26.0  208.28   245.0          Kansas   
                   PG             PG  25.0  193.04   195.0        Kentucky   
                   SF             SF  23.0  203.20   198.0      Georgetown   
                   SG             SG  22.0  195.58   207.0         Florida   

                                 Salary  
Team               Position              
Atlanta Hawks      C         12000000.0  
                   PF        18671659.0  
                   PG         8000000.0  
                   SF         4000000.0  
                   SG         5746479.0  
...                                 ...  
Washington Wizards C         13000000.0  
                   PF         8000000.0  
                   PG        15851950.0  
                   SF         4662960.0  
                   SG         5694674.0  

[149 rows x 9 columns]